In [1]:
from src_linkedin import LinkedIn

# selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# other
import pandas as pd
import time
import os

# credentials
email = 'joseph.barbierdarnal@gmail.com'
with open('../credentials.txt', 'r') as file:
    password = file.read()

In [2]:
scraper = LinkedIn()
scraper.load_page("https://www.linkedin.com/")
scraper.sleep(5)
scraper.accept_cookies()
scraper.login(email, password)

### some captcha can occurs here sometimes ):

In [3]:
scraper.load_page("https://www.linkedin.com/jobs/collections/recommended/")
scraper.close_message()

In [4]:
us_jobs = scraper.get_all_jobs(location='United States')
#southasia_jobs = scraper.get_all_jobs(location='South Asia')
#eastasia_jobs = scraper.get_all_jobs(location='East Asia')
#europe_jobs = scraper.get_all_jobs(location='European Economic Area')

[] ['3798542883', '3801279651', '3807305628', '3809926579', '3808311430', '3809535644', '3809815377', '3812445201']


IndexError: list index out of range

In [12]:
x = ['a', 'b', 'c']
y = ['a']
z = []
z[-1]

IndexError: list index out of range

In [ ]:
# concat all job dataframes
linkedin_jobs = pd.concat([us_jobs, southasia_jobs, eastasia_jobs, europe_jobs])
linkedin_jobs['website'] = 'LinkedIn'
linkedin_jobs.to_csv('../data/linkedin.csv', index=False)